# Inicialização

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import requests, csv
import settings
from combolists import ComboList
from accesslevel import AccessLevel
from dotenv import load_dotenv
import warnings
from colaborador import Colaborador
from paciente import Paciente
from acompanhante import Acompanhante
from time import sleep
from helperfunctions import helper
from database import DataBase
from estudante import Estudante
from datetime import datetime
from time import strftime, strptime
import json

warnings.filterwarnings("ignore")
load_dotenv()
settings.init()
settings.combofields = ComboList.getComboLists()
settings.accesslevels = AccessLevel.getAccessLevels()

## Código

### ESTUDANTES

In [ ]:


def Vinculo(vinculo):
    returnValue = ""

    match vinculo:
        case "VINCULO INTEGRACAO": returnValue =	"955"
        case "RJU - UFPE": returnValue =	"1"
        case "NASS - UFPE": returnValue =	"2"
        case "RESIDENTE": returnValue =	"3"
        case "EBSERH - CLT": returnValue =	"4"
        case "TERCEIRIZADO": returnValue =	"5"
        case "EBSERH - REQUISITADO": returnValue =	"6"
        case "BOLSISTA - ESTAGIARIO": returnValue =	"7"
        case "PROFESSOR": returnValue =	"8"
        case "VOLUNTÁRIO": returnValue =	"9"
        case "ESTUDANTE": returnValue =	"10"
    return returnValue

with open("teste.csv", encoding="utf-8") as arquivo_csv:
    arquivo = csv.reader(arquivo_csv, delimiter=";")
    arquivo.__next__()

    for row in arquivo:
        nome = row[0]
        exibicao = row[1]
        matricula = row[2]
        vinculo = row[3]
        cpf = row[4]
        grupo = row[5]
        curso = row[8]
        inicio_vinculo = row[6]
        fim_vinculo = row[7]

        user = {
            "CHType": 8,
            "FirstName": nome,
            "IdNumber": matricula + Vinculo(vinculo),
            "AuxText02": exibicao or None,
            "AuxText03": matricula,
            "AuxText04": cpf or None,
            "AuxDte02": datetime.strptime(inicio_vinculo, "%d/%m/%Y").isoformat(),
            "AuxDte03": datetime.strptime(fim_vinculo, "%d/%m/%Y").isoformat(),
            "CHState": 0,
            "AuxLst01": ComboList.getIndex(8, "AuxLst01", grupo),
            "AuxLst02": ComboList.getIndex(8, "AuxLst02", curso),
            "AuxLst03": None,
            "AuxLst04": ComboList.getIndex(8, "AuxLst04", vinculo),
        }
        colaboradorInvenzi = Colaborador.getColaboradorByIdNumber(
            (user["IdNumber"])
        )
        print(json.dumps(user, indent=2))
        if colaboradorInvenzi:
            # AccessLevel.addAccessLevel(colaboradorInvenzi[0]["CHID"], "Acesso estudante")
            requests.delete(f'{settings.baseUrl}/cardholders/{colaboradorInvenzi[0]["CHID"]}', verify=False)
            print(colaboradorInvenzi)
        else:
            colaborador = Estudante.createEstudante(user)
            AccessLevel.addAccessLevel(
                colaborador["CHID"], "Acesso estudante"
            )
            print(colaborador)
            # print("Colaborador não existe")